# Tensor flow and other basic stuff

In [2]:
%config IPCompleter.greedy=True
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import sklearn
from sklearn import preprocessing
from tensorflow.keras.layers.experimental import preprocessing


print(f"Tensor Flow version: {tf.__version__}")

Tensor Flow version: 2.12.0


In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pymongo import MongoClient

np.set_printoptions(precision=3, suppress=True)

def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]


def read_mongo(db="anystream", collection="statistics", query={}, host='adamkaliszan.pl', port=27017, username='anonymus', password='password', no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

In [12]:
dataFrame = read_mongo()
'''print(dataFrame)
print(str(dataFrame['system'][0]['class']))'''
classes = pd.DataFrame()
classes['a'] = dataFrame.system.apply(lambda x: x['class']['a'])
classes['arrival_stream_type'] = dataFrame.system.apply(lambda x: x['class']['arrival_stream_type'])
classes['arrival_e2d2'] = dataFrame.system.apply(lambda x: x['class']['arrival_e2d2'])
classes['service_stream_type'] = dataFrame.system.apply(lambda x: x['class']['service_stream_type'])
classes['service_e2d2'] = dataFrame.system.apply(lambda x: x['class']['service_e2d2'])

tmp = classes.groupby(['arrival_stream_type', 'arrival_e2d2'])
print(tmp.aggregate(['count']))

print(classes)
print(type(classes))


                                     a service_stream_type service_e2d2
                                 count               count        count
arrival_stream_type arrival_e2d2                                       
Gamma               1.0           5584                5584         5584
Uniform             3.0            150                 150          150
         a arrival_stream_type  arrival_e2d2 service_stream_type  service_e2d2
0      8.0             Uniform           3.0             Poisson           1.0
1      8.0             Uniform           3.0             Poisson           1.0
2      8.0             Uniform           3.0             Poisson           1.0
3      8.0             Uniform           3.0             Poisson           1.0
4      8.0             Uniform           3.0             Poisson           1.0
...    ...                 ...           ...                 ...           ...
5729  23.5               Gamma           1.0             Poisson           1.0
5730  23

In [5]:
print(dataFrame)

                                                 system   
0     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...  \
1     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...   
2     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...   
3     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...   
4     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...   
...                                                 ...   
5729  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...   
5730  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...   
5731  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...   
5732  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...   
5733  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...   

                                                   stat  
0     {'states': [{'p': 0.07650509235411221, 'out_ne...  
1     {'states': [{'p': 0.06553723807425986, 'out_ne...  
2     {'states': [{'p': 0.08318307241254727, 'out_ne...  
3     {'states': [{'p': 0.008908421008319888, 'out_n...  
4

In [33]:
groupedFrame = dataFrame.groupby(['system'])
print(groupedFrame.type)
print(groupedFrame)

AttributeError: 'DataFrameGroupBy' object has no attribute 'type'

In [42]:
asd = dataFrame[['system']].apply(lambda x: x.astype(str))
asd = asd.rename(columns= {'system' : 'system_str'})
print(asd);
'''asd.add(dataFrame)'''
'''print(asd.groupby('system')['system'].aggregate(['count']))'''


                                             system_str
0     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...
1     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...
2     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...
3     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...
4     {'class': {'a': 8.0, 'mu': 1.0, 'arrival_strea...
...                                                 ...
5699  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...
5700  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...
5701  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...
5702  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...
5703  {'class': {'a': 23.5, 'mu': 1.0, 'arrival_stre...

[5704 rows x 1 columns]


"print(asd.groupby('system')['system'].aggregate(['count']))"

## Załadowanie danych z 60 symulacji

In [4]:
#occupancyDistribution = pd.read_csv("./results.csv", "\t")
occupancyDistribution = pd.read_csv("./resultsGamma_V10.csv", "\t")

print(occupancyDistribution.columns)
print(occupancyDistribution)
print(occupancyDistribution.describe())


TypeError: read_csv() takes 1 positional argument but 2 were given

In [ ]:
V=10
SR=10
inputColNames  = [f"{l}({x})_{v}" for l in ['a', 's'] for v in range(1, V+1) if v%SR ==0 for x in range(0, v+1)]
labelColNames = [f"{l}({x})_{v}" for l in ['a'] for v in range(1, V+1) if v%SR !=0 for x in range(0, v+1)]

print("inputColNames:", inputColNames)
print("labelColNames:", labelColNames)

input = occupancyDistribution[inputColNames].copy().astype(np.float32)
label = occupancyDistribution[labelColNames].copy().astype(np.float32)

print(f"Input shape {input.shape}")
print(f"Label shape {label.shape}")

input.head()
#plt.plot(input.to_numpy()[0:17,0:6])

inputColNames: ['a(0)_10', 'a(1)_10', 'a(2)_10', 'a(3)_10', 'a(4)_10', 'a(5)_10', 'a(6)_10', 'a(7)_10', 'a(8)_10', 'a(9)_10', 'a(10)_10', 's(0)_10', 's(1)_10', 's(2)_10', 's(3)_10', 's(4)_10', 's(5)_10', 's(6)_10', 's(7)_10', 's(8)_10', 's(9)_10', 's(10)_10']
labelColNames: ['a(0)_1', 'a(1)_1', 'a(0)_2', 'a(1)_2', 'a(2)_2', 'a(0)_3', 'a(1)_3', 'a(2)_3', 'a(3)_3', 'a(0)_4', 'a(1)_4', 'a(2)_4', 'a(3)_4', 'a(4)_4', 'a(0)_5', 'a(1)_5', 'a(2)_5', 'a(3)_5', 'a(4)_5', 'a(5)_5', 'a(0)_6', 'a(1)_6', 'a(2)_6', 'a(3)_6', 'a(4)_6', 'a(5)_6', 'a(6)_6', 'a(0)_7', 'a(1)_7', 'a(2)_7', 'a(3)_7', 'a(4)_7', 'a(5)_7', 'a(6)_7', 'a(7)_7', 'a(0)_8', 'a(1)_8', 'a(2)_8', 'a(3)_8', 'a(4)_8', 'a(5)_8', 'a(6)_8', 'a(7)_8', 'a(8)_8', 'a(0)_9', 'a(1)_9', 'a(2)_9', 'a(3)_9', 'a(4)_9', 'a(5)_9', 'a(6)_9', 'a(7)_9', 'a(8)_9', 'a(9)_9']
Input shape (630, 22)
Label shape (630, 54)


,a(0)_10,a(1)_10,a(2)_10,a(3)_10,a(4)_10,a(5)_10,a(6)_10,a(7)_10,a(8)_10,a(9)_10,...,s(1)_10,s(2)_10,s(3)_10,s(4)_10,s(5)_10,s(6)_10,s(7)_10,s(8)_10,s(9)_10,s(10)_10
0,4.997979,4.998827,4.997750,4.999627,5.000669,4.996797,4.995011,4.999054,4.999118,4.997772,...,0.999995,1.997660,3.001118,3.999315,4.998825,6.001583,7.001068,8.006833,8.993415,10.009018
1,5.490233,5.499823,5.506721,5.497864,5.500509,5.498470,5.498195,5.500711,5.500023,5.498466,...,1.000545,1.998775,2.998182,3.999285,4.998265,6.001042,7.002124,7.994497,9.008581,10.000951
2,5.985474,5.993181,5.997361,5.997668,5.998716,6.001694,5.998439,6.001486,6.001760,5.994999,...,0.999076,1.996307,3.000478,4.000061,5.001675,6.001321,7.000861,7.997693,9.000064,9.999695
3,6.486227,6.502338,6.502121,6.500687,6.496743,6.498294,6.501260,6.501489,6.495543,6.500004,...,0.994825,1.997501,3.000973,3.999468,4.996759,6.003747,7.005655,7.998842,9.003159,10.000935
4,6.981313,7.012771,7.006184,7.007293,6.993352,7.003760,6.999850,7.002643,7.001285,7.003886,...,1.004527,2.004453,3.003385,3.999322,5.001534,5.996248,6.998580,8.001925,9.000112,10.000605


## Wykresy ##
Intensywność napływu zgłoszeń (Oś Y) w zależności od stanu (oś X).
Widać złą jakość symulacji

In [ ]:
%matplotlib
LambdaColNames = [f"a({x})_{v}" for v in range(1, V+1) for x in range(0, v+1)]
lambdas = occupancyDistribution[LambdaColNames].copy().astype(np.float32)
#plt.plot(lambdas.to_numpy()[0, 0:100], [x for x in range(0, 100)])

fig = plt.figure()
rowOffset = 0
rowSkip = 3
for plotNo in range (1, 11):
    ax = fig.add_subplot(2, 5, plotNo)
    offset = 0
    for v in range (1, 11):
        ax.plot([x for x in range(0, v+1)], lambdas.to_numpy()[(plotNo-1) * rowSkip + rowOffset, offset:offset+v+1])
        offset = offset + v + 1


Using matplotlib backend: TkAgg


In [ ]:
from sklearn.model_selection import train_test_split
input_train, input_test, label_train, label_test = train_test_split(input, label, test_size=0.33, random_state=42)

## Tworzenie modelu

In [ ]:
model = tf.keras.Sequential([  
  tf.keras.layers.Dense(22),
  tf.keras.layers.Dense(540, activation='relu'),
  tf.keras.layers.Dense(2160, activation='relu'),
  tf.keras.layers.Dense(540, activation='relu'),
  tf.keras.layers.Dense(54),
])

loss_fn = tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(input_train.to_numpy(), label_train.to_numpy(), epochs=30)

Epoch 1/30
14/14 [==============================] - 0s 7ms/step - loss: 82.3796 - accuracy: 0.1872
Epoch 2/30
14/14 [==============================] - 0s 7ms/step - loss: 9.4937 - accuracy: 0.1374
Epoch 3/30
14/14 [==============================] - 0s 7ms/step - loss: 3.4507 - accuracy: 0.1825
Epoch 4/30
14/14 [==============================] - 0s 7ms/step - loss: 1.6579 - accuracy: 0.2962
Epoch 5/30
14/14 [==============================] - 0s 7ms/step - loss: 1.2006 - accuracy: 0.2014
Epoch 6/30
14/14 [==============================] - 0s 7ms/step - loss: 0.9653 - accuracy: 0.2583
Epoch 7/30
14/14 [==============================] - 0s 7ms/step - loss: 0.8201 - accuracy: 0.2678
Epoch 8/30
14/14 [==============================] - 0s 7ms/step - loss: 0.6606 - accuracy: 0.2536
Epoch 9/30
14/14 [==============================] - 0s 7ms/step - loss: 0.6172 - accuracy: 0.3104
Epoch 10/30
14/14 [==============================] - 0s 7ms/step - loss: 0.4713 - accuracy: 0.2654
Epoch 11/30
14/14 

## Weryfikacja

In [ ]:
model.evaluate(input_test, label_test, verbose=2)

3/3 - 0s - loss: 0.0855 - accuracy: 0.8023


[0.08552196621894836, 0.8023256063461304]

In [ ]:
input_test

In [ ]:
model = tf.keras.Sequential([  
  tf.keras.layers.Reshape([11,2]),
  tf.keras.layers.Conv1D(filters=16, kernel_size=3, padding='same', activation='relu', input_shape=(11,2)),
  tf.keras.layers.Dense(188),
  tf.keras.layers.Flatten(),

#  tf.keras.layers.Dropout(0.02),
#  tf.keras.layers.Dense(140),
  tf.keras.layers.Dense(662),
#  tf.keras.layers.Dropout(0.01),
  tf.keras.layers.Dense(54),
])

loss_fn = tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error")
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model.fit(input_train.to_numpy(), label_train.to_numpy(), epochs=30)